# EXTRACTIVE TEXT SUMMARIZATION: "REFRESH" MODEL

## Introduction

- This document gives the interactive access to functionality of the project. One can put the text files into an input directory 
- Please follow the instruction and ensure that your files are UTF-8 encoded

### PLEASE WAIT UNTIL THE CELL BELOW STOPS RUNNING...PREPARING THE MODEL

In [1]:
from IPython.display import HTML
import my_flags

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>''')


In [2]:
import tensorflow as tf
import os

tf.app.flags.DEFINE_string("f", "", "kernel")

In [3]:
import glob
import os
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import ipywidgets as widgets
from IPython.display import display
from functools import reduce
from data_utils import DataProcessor
import tensorflow as tf

#tokenizer.tokenize('Fig. 2 shows a U.S.A. map.')
#tf.app.flags.DEFINE_string("f", "", "kernel")

def split_text(text):
    punkt_param = PunktParameters()
    abbreviation = ['u.s.a', 'fig', 'ph.d', 'ltd', 'u.s', 'corp', 'plc', 'co', 'inc',\
               'dr', 'ms', 'mr', 'mrs', 'jan', 'feb', 'mar', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov', 'dec',\
               'a.m', 'p.m', 'e.g', 'i.e', 'u.k']

    punkt_param.abbrev_types = set(abbreviation)
    tokenizer = PunktSentenceTokenizer(punkt_param)
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    spl = tokenizer.tokenize(text)
    out = []
    i = 0
    for s in spl:
        i+=1
        
        if s[-1] == '.':
            s = s[:-1] + ' .'
        
        s = s.replace('“', '"')
        s = s.replace('”', '"')
        s = s.replace('."', ' . " ')
        s = s.replace('"', ' " ')
        s = s.replace(',', ' , ')
        s = s.replace(':', ' : ')
        s = s.replace(';', ' ; ')
        s = s.replace('’', "'")
        s = s.replace("'s", " 's" )
        s = s.replace("n't", " not" )
        s = s.replace("'re", " 're" )
        #print(s+'\n')
        #print(i, s)
        out.append(s)
        
    return out
        



address = widgets.Text(
    value='my_input_directory',
    placeholder='',
    description='Input dir:',
    disabled=False
)

display(address)

message = widgets.Textarea(
    value='',
    placeholder='',
    description='',
    disabled=True
    
)  

display(message)

button_validate_split = widgets.Button(
    value=False,
    description='Validate the split',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check'
)

reset_input = widgets.Button(
    value=False,
    description='Reset input files...',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check',

)

display(reset_input)
reset_input.layout.visibility = 'hidden'

display(button_validate_split)
button_validate_split.layout.visibility = 'hidden'

def handle_add(sender):
    reset_input.layout.visibility = 'hidden'
    to_dir = "data/bloomberg/test/mainbody"
    ld = lambda x,y: x + '\n' + y
    
    for file in os.listdir(address.value):
        #print('\n' + file)
        if file.endswith(".txt"):
            with open(address.value + "/" + str(file), "r+", encoding = "UTF-8") as ff:
                ss = split_text(ff.read())
                ss = reduce(ld, ss)
                with open(to_dir + "/" + str(file), "w+", encoding = "UTF-8") as fl:
                    fl.write(ss)
                    
    message.value = "Please verify the split into sentences in the editor, correct it if nesessary and validate it."
    message.disabled = False
    for file in os.listdir(address.value):
        os.system("start notepad++ "+ to_dir + "/" + str(file))

    button_validate_split.layout.visibility = 'visible'
    
    
    
def on_c(sender):#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    reset_input.layout.visibility = 'hidden'
    message.value = "Please wait until the end of summarization. This may take several minutes..."
    print("Loading the embedding...")
    vocab_dict, word_embedding_array = DataProcessor().prepare_vocab_embeddingdict()
    
    def find_sentences(s):
        #print(s)
        previous = 0
        L = []
        for i, word in enumerate(s):
            if (word == '.'):
                L.append(s[previous + 1:i + 1])
                previous = i
        return L
    
    def print_sentences(L, f):
        s = f + "\n"
        #print(f)
        for sentence in L:

            for x in sentence.split(): 
                try:
                    s += str(vocab_dict[x]) + " "
                except KeyError:
                    s += "1 "
            s += '\n'
        s += "\n"
        return s
    
    
    def preprocess(dirr_from, dirr_to):    
        s = ""
        import os
        for file in os.listdir(dirr_from):
            if file.endswith(".txt"):
                with open(dirr_from + "/" + str(file), "r", encoding = "UTF-8") as ff:
                    q = print_sentences(ff.read().split('\n'),"blmbrg-" + file[:-4])

                    s += q

        with open(dirr_to + "/" + "bloomberg.test.doc","w+") as f:
            f.write(s)
        
        
    fr = "data/bloomberg/test/mainbody"
    to = "data/mypreprocdir"
    preprocess(fr, to)
    
    print("Starting summarization...")
    #os.system("python document_summarizer_training_testing.py")
    #subprocess.call(["python document_summarizer_training_testing.py"])
    filename = "document_summarizer_training_testing.py"
    %run $filename > file
    #%load ./document_summarizer_training_testing.py
    
    dr =  "trainingdir/model.ckpt.epoch-"+ str(my_flags.FLAGS.model_to_load) + ".test.predictions"
    
    print("Writing summaries...")
    
    def print_summaries(dr):
        s = {}
        q = []

        sums = []
        with open(dr, "r", encoding = 'UTF-8') as f:   
            sums = f.read().split("\n\n")

        summs = {x[0][7:]:[l.split('\t')[1] for l in x[1:]] for x in [z.split('\n') for z in sums]}

        lexp = lambda x,y: x + '\n' + y
        ''''''

        for file in os.listdir("data/bloomberg/test/mainbody"):
            if file.endswith(".txt"):
                with open("data/bloomberg/test/mainbody/" + str(file), "r+", encoding = "UTF-8") as ff:
                    q = ff.read().split('\n')
                #print(len(q), len(summs[file[:-4]]))
                assert(len(q) == len(summs[file[:-4]]))
                s[file[:-4]] = reduce(lexp, [str(x) + ' ' + y for x,y, in zip(summs[file[:-4]], q)] )

        return s
    
    
    s = ''
    for file, text in print_summaries(dr).items():
        s += file + '\n' + text + '\n\n'
    
    
    with open("summaries.txt", "w+", encoding = 'UTF-8') as f:
        f.write(s)
    
    tf.reset_default_graph() 


    for file in os.listdir("data/bloomberg/test/mainbody"):
        os.remove("data/bloomberg/test/mainbody/" + file)
    #print_summaries(dr)
    message.value = "Check the results in <summary.txt> in working directory. You can now restart the procedure.\
        If you want to add new articles please update if necessary input directory and press 'Reset input'... "
    reset_input.layout.visibility = 'visible'
    
    
addr = address.on_submit(handle_add)

reset_input.on_click(handle_add)

button_validate_split.on_click(on_c)

message.value = "Enter the path of your input directory and press <Enter>..."
##################################################

#split_text(open('try.txt', 'r').read(), tokenizer)

Text(value='my_input_directory', description='Input dir:', placeholder='')

Textarea(value='', disabled=True, placeholder='')

Button(button_style='warning', description='Reset input files...', icon='check', style=ButtonStyle())

Button(button_style='success', description='Validate the split', icon='check', style=ButtonStyle())

Loading the embedding...
Reading pretrained word embeddings file: data/1-billion-word-language-modeling-benchmark-r13output.word2vec.vec
0 ...
100000 ...
200000 ...
300000 ...
400000 ...
500000 ...
Read pretrained embeddings: (559183, 200)
Size of vocab: 559185 (_PAD:0, _UNK:1)
Writing vocab file: trainingdir/vocab.txt
Starting summarization...
Prepare vocab dict and read pretrained word embeddings ...
Reading pretrained word embeddings file: data/1-billion-word-language-modeling-benchmark-r13output.word2vec.vec
0 ...
100000 ...
200000 ...
300000 ...
400000 ...
500000 ...
Read pretrained embeddings: (559183, 200)
Size of vocab: 559185 (_PAD:0, _UNK:1)
Writing vocab file: trainingdir/vocab.txt
Prepare test data ...
Data file prefix (.doc, .title, .image, .label.multipleoracle): data/mypreprocdir/bloomberg.test
Data sizes: 2 
Reading data (no padding to save memory) ...
0 ...
Writing data files with prefix (.filename, .doc, .title, .image, .label, .weight, .rewards): trainingdir/bloomber